### Data Preprocess

In [8]:
import pandas as pd
import numpy as np
import re

In [9]:
# custom character alphabet for word
tr = re.compile(r"[abcçdefgğhıijklmnoöprsştuüvyzqxw]+", re.IGNORECASE|re.UNICODE) # Turkish filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789'
en = re.compile(r"[abcdefghıijklmnopqrstxuvwyz]+", re.IGNORECASE|re.UNICODE) # English
nl = re.compile(r"[abcdefghıijklmnopqrstxuvwyzāăēĕīĭōŏūŭ]+", re.IGNORECASE|re.UNICODE) # Dutch (Flemenk)
fr = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàâæèéêëîïôœùûüÿ]+", re.IGNORECASE|re.UNICODE) # French
de = re.compile(r"[abcdefghıijklmnopqrstxuvwyzäöüß]+", re.IGNORECASE|re.UNICODE) # Deutch
es = re.compile(r"[abcdefghıijklmnopqrstxuvwyzñáéíóú]+", re.IGNORECASE|re.UNICODE) # Spanish (¿¡)
pt = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàáâãéêíóôõú]+", re.IGNORECASE|re.UNICODE) # Portuguese
it = re.compile(r"[abcdefghıijklmnopqrstxuvwyzàéèìòùî]+", re.IGNORECASE|re.UNICODE) # Italian
ar = re.compile(r"[ٿصؼۤڳڲؿڎػڠجڿ٬ٸؽؒؓطۄڀۂؘؔتٚڛےٝڜؖڦ٫ډ۰زۇٖۀ،لۓعٮێڔ۶ؚۧۜڤۏإٞٷؗۖ؈ژۣؕؑٴأۻڸۺگاڴڹۯ؉ْڌ؍ي؟ـٟړۅؐڶُىڽېًۢؠضۚڄٛڏٱۦ٩س٦ڼڂٔۘ٠ښٌٍ۬ٳ۾ٲږذۋٵٜ٘ڞڅںٗهڣۿپڒۥۗڋیؙم؞ثۨٹڵڪظٶۭ١ڭەڨحٕ؎ٺڷٰ۪۫ڻڥۛڑڟټآڡغګ؊ّٯڧڮ؏ۮ؋ؤ٪ؾڗۼق۟دکوِڰڐۃ۽ہفرڇچڝ۴بۈٽڕۡھةٓڃئ؛ڬٙڙڢڱۊَۆۉځ۠ۍۑۙڊنءڈٻشڍ؇۵كخ\ا]+", re.IGNORECASE|re.UNICODE) # Arabic

In [10]:
def clean_text(text):
    text_clean = re.findall(custom_alp, text)
    text_result = " ".join(text_clean)
    return text_result

In [11]:
folder_name = "Turkish"
data_kind = "sentence" # twogram, fourgram
spec_folder = "Sentence" # Sentence, N Gram
custom_alp = tr

In [6]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Result/Word/Merge/Word_Merge2.xlsx")
#df_word = pd.read_csv("")
df_word


,word,frequency
0,bir,18835735
1,fikret,12101580
2,bu,11062659
3,ne,8025880
4,ve,7766036
...,...,...
164296,buldugumuzda,43
164297,boşandıklarını,42
164298,endişenlenme,42
164299,uzaylıymışım,42


In [7]:
df_word.iloc[:,0] = df_word.iloc[:,0].apply(lambda x: clean_text(x))
df_word.dropna(inplace=True)
df_word.drop(df_word[df_word.iloc[:,0] == ""].index, inplace=True)
df_word.reset_index(drop=True, inplace=True)
df_word

,word,frequency
0,bir,18835735
1,fikret,12101580
2,bu,11062659
3,ne,8025880
4,ve,7766036
...,...,...
164296,buldugumuzda,43
164297,boşandıklarını,42
164298,endişenlenme,42
164299,uzaylıymışım,42


In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
word_num = 1000
df_word = df_word.head(word_num)
df_word

In [ ]:
word_list = df_word["word"].values.tolist()

In [ ]:
df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Result/{spec_folder.lower().capitalize()}/Merge/{data_kind.lower().capitalize()}_Merge2_Code.csv") # sentence, n gram dataframe
df

In [ ]:
d_list  = df.iloc[:,0].values.tolist()

In [ ]:
resultlist = []

In [ ]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
resultlist = manager.list()

def word_in_wordgroup(d_list):
    mergelist = []
    try:
        word = d_list.split()
    except:
        pass
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist.append(d_list)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list) # string_word liste

In [ ]:
result_list = list(resultlist)
df_result = pd.DataFrame(result_list)
df_result = df_result.rename(columns = {0: f"{data_kind}"})
df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind}")
df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
df_merge_result

In [ ]:
df_merge_result.to_csv(f"{data_kind.capitalize()}_With_{word_num}_Word.csv", index=False)

In [ ]:
df_word.to_excel(f"Word_{word_num}.xlsx", sheet_name='Word', index=False)